# ***High-level API Index***

## ***Lists of all high-level async/await enabled asyncio APIs.***

# ***Asyncio***

Asyncio is the standard library package with Python that aims to help you write asynchronous code by giving you an easy way to write, execute, and structure your coroutines. The Asyncio library is for concurrency, which is not to be confused with parallelism.


asyncio is often a perfect fit for IO-bound and high-level structured network code.

asyncio is used as a foundation for multiple Python asynchronous frameworks that provide high-performance network and web-servers, database connection libraries, distributed task queues, etc.

The asyncio module provides a framework that revolves around the event loop. An event loop basically waits for something to happen and then acts on the event. It is responsible for handling such things as I/O and system events.

asyncio actually has several loop implementations available to it. The module will default to the one most likely to be the most efficient for the operating system it is running under; however you can explicitly choose the event loop if you so desire. An event loop basically says “when event A happens, react with function B”.

 Asyncio allows you to easily write single-threaded concurrent programs that utilize something called coroutines, these coroutines are like a stripped down threads and don’t come with the same inherit performance issues that your full-fat threads would typically come with.

## ***Coroutines***

So these coroutines are essentially lightweight versions of your more traditional threads. By using these we essentially enable ourselves to write asynchronous programs that are very similar to threads but they run on top of a single thread. We can define coroutines in 2 distinct ways.

In [4]:
import asyncio

async def myFunc1():
    print("Coroutine 1")

#@asyncio.coroutine
async def myFunc2():
    print("Coroutine 2")

### ***Tasks***

Utilities to run asyncio programs, create Tasks, and await on multiple things with timeouts

### ***1.run()***

 Create event loop, run a coroutine, close the loop.

This function runs the passed coroutine, taking care of managing the asyncio event loop, finalizing asynchronous generators, and closing the threadpool.

This function cannot be called when another asyncio event loop is running in the same thread.

If debug is True, the event loop will be run in debug mode.

This function always creates a new event loop and closes it at the end. It should be used as a main entry point for asyncio programs, and should ideally only be called once.

### syntax: asyncio.run(coro, *, debug=False)

***Normal function***

In [2]:
import asyncio,time

def main():
  print('hello')
  time.sleep(2)
  print('world')

main()

hello
world


***Example: Asyncio Function***

In [3]:
import asyncio

async def main():
  print('hello')
  await asyncio.sleep(2)
  print('world')

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")

    await say_after(1, 'hello')
    await say_after(2, 'world')

    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())

started at 11:09:23
hello
world
finished at 11:09:26


### ***2. create_task()***

Start an asyncio Task.

Wrap the coro coroutine into a Task and schedule its execution. Return the Task object.

If name is not None, it is set as the name of the task using Task.set_name().

The task is executed in the loop returned by get_running_loop(), RuntimeError is raised if there is no running loop in current thread.

In [ ]:
import asyncio

async def coro():
  print("hello jack")

# In Python 3.7+
task = asyncio.create_task(coro())

# This works in all Python versions but is less readable
task = asyncio.ensure_future(coro())

hello jack
hello jack


### ***3. await sleep()***

Sleep for a number of seconds.

If result is provided, it is returned to the caller when the coroutine completes.

sleep() always suspends the current task, allowing other tasks to run.

Setting the delay to 0 provides an optimized path to allow other tasks to run. This can be used by long-running functions to avoid blocking the event loop for the full duration of the function call.

***Example of coroutine displaying the current date every second for 5 seconds:***

In [ ]:
import asyncio
import datetime

async def display_date():
    loop = asyncio.get_running_loop()
    end_time = loop.time() + 5.0
    while True:
        print(datetime.datetime.now())
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(1)

asyncio.run(display_date())

2022-04-28 11:10:08.512407
2022-04-28 11:10:09.513913
2022-04-28 11:10:10.515447
2022-04-28 11:10:11.516928
2022-04-28 11:10:12.518993


### ***4. await gather()***

Schedule and wait for things concurrently.

If return_exceptions is False (default), the first raised exception is immediately propagated to the task that awaits on gather(). Other awaitables in the aws sequence won’t be cancelled and will continue to run.

If return_exceptions is True, exceptions are treated the same as successful results, and aggregated in the result list.

If gather() is cancelled, all submitted awaitables (that have not completed yet) are also cancelled.

If any Task or Future from the aws sequence is cancelled, it is treated as if it raised CancelledError – the gather() call is not cancelled in this case. This is to prevent the cancellation of one submitted Task/Future to cause other Tasks/Futures to be cancelled.

*** Removed the loop parameter.***

In [ ]:
import asyncio

async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    return f

async def main():
    # Schedule three calls *concurrently*:
    L = await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    print(L)

asyncio.run(main())


Task A: Compute factorial(2), currently i=2...
Task B: Compute factorial(3), currently i=2...
Task C: Compute factorial(4), currently i=2...
Task A: factorial(2) = 2
Task B: Compute factorial(3), currently i=3...
Task C: Compute factorial(4), currently i=3...
Task B: factorial(3) = 6
Task C: Compute factorial(4), currently i=4...
Task C: factorial(4) = 24
[2, 6, 24]


 If return_exceptions is False, cancelling gather() after it has been marked done won’t cancel any submitted awaitables. For instance, gather can be marked done after propagating an exception to the caller, therefore, calling gather.cancel() after catching an exception (raised by one of the awaitables) from gather won’t cancel any other awaitables.

***5. await wait_for()***

To avoid the task cancellation, wrap it in shield().

The function will wait until the future is actually cancelled, so the total wait time may exceed the timeout. If an exception happens during cancellation, it is propagated.

If the wait is cancelled, the future aw is also cancelled.

### ***Example***

In [ ]:
import asyncio
async def eternity():
    # Sleep for one hour
    await asyncio.sleep(3600)
    print('yay!')

async def main():
    # Wait for at most 1 second
    try:
        await asyncio.wait_for(eternity(), timeout=3.0)
    except asyncio.TimeoutError:
        print('timeout!')

asyncio.run(main())


timeout!


# ***5. await shield(aw)***

If aw is a coroutine it is automatically scheduled as a Task.

The statement:

***res = await shield(something())***

is equivalent to:

***res = await something()***

except that if the coroutine containing it is cancelled, the Task running in something() is not cancelled. From the point of view of something(), the cancellation did not happen. Although its caller is still cancelled, so the “await” expression still raises a CancelledError.

If something() is cancelled by other means (i.e. from within itself) that would also cancel shield().

If it is desired to completely ignore cancellation (not recommended) the shield() function should be combined with a try/except clause, as follows:

***get_event_loop()***


The main component of any asyncio based Python program has to be the underlying event loop.

**Let’s take a quick look at how you can define a very simple event loop. In order to instantiate an event loop we’ll use asyncio.get_event_loop(), we’ll then start a try... finally and within the body of our try we’ll specify that we want our newly instantiated event loop to run until it has completed our myCoroutine() function.**

In [ ]:
import asyncio
import time

async def myWork():
  print("Starting Work")
  time.sleep(3)
  print("Finishing Work")

loop = asyncio.get_event_loop()
try:
  loop.run_until_complete(myWork())
  print("jack")
finally:
  print("exit from the  loop")

Starting Work
Finishing Work
jack
exit from the  loop


In [ ]:
import asyncio

async def count_number(n): #count the number
  for i in range(n):
    print(i)
    await asyncio.sleep(3)

async def call_shield():
  try:
    await asyncio.shield(count_number(5))
  except asyncio.CancelledError:
    print('This gets called at 3, not 5')
  return "write something"

async def shield_sleep(info):
  await asyncio.sleep(3)
  info.cancel()

async def main():
  info = asyncio.ensure_future(call_shield())
  ct = asyncio.ensure_future(shield_sleep(info)) #return None
  store_shield_sleep = await info
  print(store_shield_sleep)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

0
Task Executed
Task Executed
Task Executed
1
This gets called at 3, not 5
write something


In [ ]:
import asyncio
async def coro():
    print('starting')
    await asyncio.sleep(2)
    print('done sleep')

async def cancel_it(some_task):
    await asyncio.sleep(0.5)
    some_task.cancel()
    print('cancellation effected')

async def main():
    loop = asyncio.get_event_loop()
    real_task = loop.create_task(coro())
    shield = asyncio.shield(real_task)
    # cancel the shield in the background while we're waiting
    loop.create_task(cancel_it(shield))
    await real_task

    assert not real_task.cancelled()
    assert shield.cancelled()

asyncio.get_event_loop().run_until_complete(main())

starting


Exception ignored in: <coroutine object sleep at 0x7f0639942ef0>
KeyError: ('__builtins__',)
<string>:27: RuntimeWarning: coroutine 'sleep' was never awaited


cancellation effected
done sleep


### ***6. await wait()***

	
Monitor for completion.

### Syntax : coroutine asyncio.wait(aws, *, timeout=None, return_when=ALL_COMPLETED)

Returns two sets of Tasks/Futures: (done, pending).

Run awaitable objects in the aws iterable concurrently and block until the condition specified by return_when.

The aws iterable must not be empty.

Returns two sets of Tasks/Futures: (done, pending).

### Usage:

***done, pending = await asyncio.wait(aws)***

timeout (a float or int), if specified, can be used to control the maximum number of seconds to wait before returning.

Note that this function does not raise asyncio.TimeoutError. Futures or Tasks that aren’t done when the timeout occurs are simply returned in the second set.

return_when indicates when this function should return. It must be one of the following constants:

Unlike wait_for(), wait() does not cancel the futures when a timeout occurs.

In [ ]:
import asyncio,random
async def coro(tag):
    print(">", tag)
    await asyncio.sleep(random.uniform(0.5, 5))
    print("<", tag)
    return tag


loop = asyncio.get_event_loop() #You've already read the above (await shield(aw)) page.

tasks = [coro(i) for i in range(1, 11)]

print("Get first result:")
finished, unfinished = loop.run_until_complete(
    asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED))

for task in finished:
    print(task.result())
print("unfinished:", len(unfinished))

print("Get more results in 2 seconds:")
finished2, unfinished2 = loop.run_until_complete(
    asyncio.wait(unfinished, timeout=2))

for task in finished2:
    print(task.result())
print("unfinished2:", len(unfinished2))

print("Get all other results:")
finished3, unfinished3 = loop.run_until_complete(asyncio.wait(unfinished2))

for task in finished3:
    print(task.result())

print("exit from the  loop")

Get first result:
> 3
> 10
> 1
> 2
> 9
> 7
> 6
> 5
> 4
> 8
< 6
6
unfinished: 9
Get more results in 2 seconds:
< 7
< 10
< 8
< 9
< 4
9
8
7
4
10
unfinished2: 4
Get all other results:
< 2
< 1
< 5
< 3
1
2
5
3
exit from the  loop


### ***7. current_task()***

	
Return the current Task.

***syntax : asyncio.current_task(loop=None)***

Return the currently running Task instance, or None if no task is running.

If loop is None get_running_loop() is used to get the current loop.

### ***8.all_tasks()***

Return all tasks for an event loop.

**asyncio.all_tasks(loop=None)**

Being able to ascertain what tasks are currently pending can be important for systems in production needing to be able to anticipate things such as workload etc. The all_tasks() method gives us some incite as to what tasks are currently in a pending state before they are executed by our event loop.

In [ ]:
import asyncio
import time

async def myTask():
    time.sleep(1)
    print("Processing Task")

async def main():
    for i in range(5):
        asyncio.ensure_future(myTask())
    pending = asyncio.Task.all_tasks()
    print(pending)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())
print("Completed All Tasks")
print("close loop")

{<Task pending coro=<myTask() running at <ipython-input-122-930d022e314c>:4>>, <Task finished coro=<coro() done, defined at <ipython-input-121-bb5eafd543f5>:2> result=5>, <Task finished coro=<sleep() done, defined at /usr/lib/python3.7/asyncio/tasks.py:582> result=3>, <Task finished coro=<coro() done, defined at <ipython-input-121-bb5eafd543f5>:2> result=1>, <Task pending coro=<myTask() running at <ipython-input-122-930d022e314c>:4>>, <Task finished coro=<coro() done, defined at <ipython-input-121-bb5eafd543f5>:2> result=8>, <Task finished coro=<main() done, defined at <ipython-input-26-7af1c80f4abc>:3> result=None>, <Task finished coro=<tcp_echo_client() done, defined at <ipython-input-55-163cf17985ff>:3> exception=ConnectionRefusedError(111, "Connect call failed ('127.0.0.1', 8000)")>, <Task finished coro=<coro() done, defined at <ipython-input-121-bb5eafd543f5>:2> result=10>, <Task finished coro=<main() done, defined at <ipython-input-27-fd0c48040b80>:3> result=None>, <Task pending 

Return a set of not yet finished Task objects run by the loop.

If loop is None, get_running_loop() is used for getting current loop.

### ***9. Task***

	
Task object.

### syntax : class asyncio.Task(coro, *, loop=None, name=None)

Tasks are used to run coroutines in event loops. If a coroutine awaits on a Future, the Task suspends the execution of the coroutine and waits for the completion of the Future. When the Future is done, the execution of the wrapped coroutine resumes.

Event loops use cooperative scheduling: an event loop runs one Task at a time. While a Task awaits for the completion of a Future, the event loop runs other Tasks, callbacks, or performs IO operations.

Use the high-level asyncio.create_task() function to create Tasks, or the low-level loop.create_task() or ensure_future() functions. Manual instantiation of Tasks is discouraged.

To cancel a running Task use the cancel() method. Calling it will cause the Task to throw a CancelledError exception into the wrapped coroutine. If a coroutine is awaiting on a Future object during cancellation, the Future object will be cancelled.

***The following example illustrates how coroutines can intercept the cancellation request:***

In [ ]:
import asyncio,time
async def cancel_me():
    print('cancel_me(): before sleep')

    try:
        # Wait for 1 hour
        await asyncio.sleep(3600)
    except asyncio.CancelledError:
        print('cancel_me(): cancel sleep')
        raise
    finally:
        print('cancel_me(): after sleep')

async def main():
    # Create a "cancel_me" Task
    task = asyncio.create_task(cancel_me())

    # Wait for 1 second
    await asyncio.sleep(1)

    task.cancel()
    try:
        await task
    except asyncio.CancelledError:
        print("main(): cancel_me is cancelled now")

asyncio.run(main())

cancel_me(): before sleep
cancel_me(): cancel sleep
cancel_me(): after sleep
main(): cancel_me is cancelled now


### ***10. to_thread()***



	
Asynchronously run a function in a separate OS thread.

### syntax : coroutine asyncio.to_thread(func, /, *args, **kwargs)

Asynchronously run function func in a separate thread.

Any *args and **kwargs supplied for this function are directly passed to func. Also, the current contextvars.Context is propagated, allowing context variables from the event loop thread to be accessed in the separate thread.

Return a coroutine that can be awaited to get the eventual result of func.

This coroutine function is primarily intended to be used for executing IO-bound functions/methods that would otherwise block the event loop if they were ran in the main thread. For example:

***to_thread is only available in python 3.9+, if you are working with python 3.8 or an older version***

In [ ]:
import asyncio,time
def blocking_io():
    print(f"start blocking_io at {time.strftime('%X')}")
    # Note that time.sleep() can be replaced with any blocking
    # IO-bound operation, such as file operations.
    time.sleep(1)
    print(f"blocking_io complete at {time.strftime('%X')}")

async def main():
    print(f"started main at {time.strftime('%X')}")

    await asyncio.gather(
        asyncio.to_thread(blocking_io),
        asyncio.sleep(1))

    print(f"finished main at {time.strftime('%X')}")


asyncio.run(main())

Directly calling blocking_io() in any coroutine would block the event loop for its duration, resulting in an additional 1 second of run time. Instead, by using asyncio.to_thread(), we can run it in a separate thread without blocking the event loop.

# ***11.run_coroutine_threadsafe()***

	
Schedule a coroutine from another OS thread.

### syntax : asyncio.run_coroutine_threadsafe(coro, loop)

***example***

In [ ]:
import asyncio
async def stop():
    await asyncio.sleep(3)

event_loop = asyncio.get_event_loop()
coro = asyncio.sleep(1, result=3)
future = asyncio.run_coroutine_threadsafe(coro, event_loop)
event_loop.run_until_complete(stop())
print(future.result())

Task Executed
Task Executed
Task Executed
3


If an exception is raised in the coroutine, the returned Future will be notified. It can also be used to cancel the task in the event loop:

# ***12. for in as_completed()***

	
Monitor for completion with a for loop.

### syntax: asyncio.as_completed(aws, *, timeout=None)

***Example:***

for coro in as_completed(aws):

    earliest_result = await coro

# ***Queues***

Queues should be used to distribute work amongst multiple asyncio Tasks, implement connection pools, and pub/sub patterns.

## ***Queue***

Queue is a FIFO queue.

### Syntax: class asyncio.Queue(maxsize=0).

A first in, first out (FIFO) queue.

If maxsize is less than or equal to zero, the queue size is infinite. If it is an integer greater than 0, then await put() blocks when the queue reaches maxsize until an item is removed by get().

Unlike the standard library threading queue, the size of the queue is always known and can be returned by calling the qsize() method.

***Queues can be used to distribute workload between several concurrent tasks:***

In [ ]:
import asyncio
import random
import time


async def worker(name, queue):
    while True:
        # Get a "work item" out of the queue.
        sleep_for = await queue.get()

        # Sleep for the "sleep_for" seconds.
        await asyncio.sleep(sleep_for)

        # Notify the queue that the "work item" has been processed.
        queue.task_done()

        print(f'{name} has slept for {sleep_for:.2f} seconds')


async def main():
    # Create a queue that we will use to store our "workload".
    queue = asyncio.Queue()

    # Generate random timings and put them into the queue.
    total_sleep_time = 0
    for _ in range(20):
        sleep_for = random.uniform(0.05, 1.0)
        total_sleep_time += sleep_for
        queue.put_nowait(sleep_for)

    # Create three worker tasks to process the queue concurrently.
    tasks = []
    for i in range(3):
        task = asyncio.create_task(worker(f'worker-{i}', queue))
        tasks.append(task)

    # Wait until the queue is fully processed.
    started_at = time.monotonic()
    await queue.join()
    total_slept_for = time.monotonic() - started_at

    # Cancel our worker tasks.
    for task in tasks:
        task.cancel()
    # Wait until all worker tasks are cancelled.
    await asyncio.gather(*tasks, return_exceptions=True)

    print('====')
    print(f'3 workers slept in parallel for {total_slept_for:.2f} seconds')
    print(f'total expected sleep time: {total_sleep_time:.2f} seconds')


asyncio.run(main())

worker-1 has slept for 0.20 seconds
worker-2 has slept for 0.31 seconds
worker-1 has slept for 0.11 seconds
worker-1 has slept for 0.19 seconds
worker-0 has slept for 0.51 seconds
worker-0 has slept for 0.34 seconds
worker-1 has slept for 0.59 seconds
worker-2 has slept for 0.87 seconds
worker-1 has slept for 0.27 seconds
worker-2 has slept for 0.23 seconds
worker-1 has slept for 0.25 seconds
worker-0 has slept for 0.90 seconds
worker-2 has slept for 0.45 seconds
worker-1 has slept for 0.30 seconds
worker-2 has slept for 0.20 seconds
worker-0 has slept for 0.63 seconds
worker-1 has slept for 0.73 seconds
worker-2 has slept for 0.59 seconds
worker-0 has slept for 0.88 seconds
worker-1 has slept for 0.97 seconds
====
3 workers slept in parallel for 3.61 seconds
total expected sleep time: 9.52 seconds


## ***PriorityQueue***

A priorityQueue is a priority queue.

### Syntax: class asyncio.PriorityQueue

A variant of Queue; retrieves entries in priority order (lowest first).

Entries are typically tuples of the form (priority_number, data).

## ***LifoQueue***

### ***Syntax : class asyncio.LifoQueue***

A variant of Queue that retrieves most recently added entries first (last in, first out).

# ***Subprocesses***

Utilities to spawn subprocesses and run shell commands.

This section describes high-level async/await asyncio APIs to create and manage subprocesses.



In [ ]:
import asyncio

async def run(cmd):
    proc = await asyncio.create_subprocess_shell(
        cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE)

    stdout, stderr = await proc.communicate()

    print(f'[{cmd!r} exited with {proc.returncode}]')
    if stdout:
        print(f'[stdout]\n{stdout.decode()}')
    if stderr:
        print(f'[stderr]\n{stderr.decode()}')

asyncio.run(run('ls /zzz'))

['ls /zzz' exited with 2]
[stderr]
ls: cannot access '/zzz': No such file or directory



### ***await create_subprocess_exec()***

### Syntax: coroutine asyncio.create_subprocess_exec(program, *args, stdin=None, stdout=None, stderr=None, limit=None, **kwds)

Create a subprocess from one or more string arguments specified by args.

args must be a list of strings represented by:

str;

or bytes, encoded to the filesystem encoding.

The first string specifies the program executable, and the remaining strings specify the arguments. Together, string arguments form the argv of the program.

This is similar to the standard library subprocess.Popen class called with shell=False and the list of strings passed as the first argument; however, where Popen takes a single argument which is list of strings, subprocess_exec takes multiple string arguments.

### ***await create_subprocess_shell()***

### Syntax: coroutine asyncio.create_subprocess_shell(cmd, stdin=None, stdout=None, stderr=None, limit=None, **kwds)

Run the cmd shell command.

The limit argument sets the buffer limit for StreamReader wrappers for Process.stdout and Process.stderr (if subprocess.PIPE is passed to stdout and stderr arguments).

Return a Process instance.

### ***Example The subprocess is created by the create_subprocess_exec() function:***

In [ ]:
import asyncio
import sys

async def get_date():
    code = 'import datetime; print(datetime.datetime.now())'

    # Create the subprocess; redirect the standard output
    # into a pipe.
    proc = await asyncio.create_subprocess_exec(
        sys.executable, '-c', code,
        stdout=asyncio.subprocess.PIPE)

    # Read one line of output.
    data = await proc.stdout.readline()
    line = data.decode('ascii').rstrip()

    # Wait for the subprocess exit.
    await proc.wait()
    return line

date = asyncio.run(get_date())
print(f"Current date: {date}")

Current date: 2022-04-28 05:07:39.165879


# ***Streams***

Streams are high-level async/await-ready primitives to work with network connections. Streams allow sending and receiving data without using callbacks or low-level protocols and transports.

### Here is an example of a TCP echo client written using asyncio streams:

In [ ]:
# WORKING ON THIS POINT
#     -------------------------------------------------
import asyncio

async def tcp_echo_client(message):
    reader, writer = await asyncio.open_connection('127.0.0.1', 8000)

    print(f'Send: {message!r}')
    writer.write(message.encode())
    await writer.drain()

    data = await reader.read(100)
    print(f'Received: {data.decode()!r}')

    print('Close the connection')
    writer.close()
    await writer.wait_closed()

asyncio.run(tcp_echo_client('Hello World!'))


#   --------------------------------------------------------

### ***await open_connection()***

Establish a TCP connection.

### Syntax: coroutine asyncio.open_connection(host=None, port=None, *, limit=None, ssl=None, family=0, proto=0, flags=0, sock=None, local_addr=None, server_hostname=None, ssl_handshake_timeout=None, happy_eyeballs_delay=None, interleave=None)

Establish a network connection and return a pair of (reader, writer) objects.

The returned reader and writer objects are instances of StreamReader and StreamWriter classes.

### ***await open_unix_connection()***

Establish a Unix socket connection.

### Syntax: coroutine asyncio.open_unix_connection(path=None, *, limit=None, ssl=None, sock=None, server_hostname=None, ssl_handshake_timeout=None) **bold text**

Establish a Unix socket connection and return a pair of (reader, writer).

Similar to open_connection() but operates on Unix sockets.

### ***await start_server()***

	
Start a TCP server.

### Syntax: coroutine asyncio.start_server(client_connected_cb, host=None, port=None, *, limit=None, family=socket.AF_UNSPEC, flags=socket.AI_PASSIVE, sock=None, backlog=100, ssl=None, reuse_address=None, reuse_port=None, ssl_handshake_timeout=None, start_serving=True)

Start a socket server.

The client_connected_cb callback is called whenever a new client connection is established. It receives a (reader, writer) pair as two arguments, instances of the StreamReader and StreamWriter classes.

client_connected_cb can be a plain callable or a coroutine function; if it is a coroutine function, it will be automatically scheduled as a Task.

limit determines the buffer size limit used by the returned StreamReader instance. By default the limit is set to 64 KiB.

### ***await start_unix_server()***

	
Start a Unix socket server.

### Syntax: coroutine asyncio.start_unix_server(client_connected_cb, path=None, *, limit=None, sock=None, backlog=100, ssl=None, ssl_handshake_timeout=None, start_serving=True) **bold text**

Start a Unix socket server.

Similar to start_server() but works with Unix sockets.

Availability: Unix.

### ***StreamReader***

High-level async/await object to receive network data.

### Syntax: class asyncio.StreamReader

Represents a reader object that provides APIs to read data from the IO stream.

It is not recommended to instantiate StreamReader objects directly; use open_connection() and start_server() instead.

### ***StreamWriter***

High-level async/await object to send network data.

### Syntax: class asyncio.StreamWriter

Represents a writer object that provides APIs to write data to the IO stream.

It is not recommended to instantiate StreamWriter objects directly; use open_connection() and start_server() instead.

**1. write(data)**

**2. writelines(data)**

# ***Synchronization***

Threading-like synchronization primitives that can be used in Tasks.

### ***Lock***

*In this example we are going to create a asyncio.Lock() instance and we are going to try to acquire this lock using with await lock. Once our worker has attained this lock we will then execute our critical section of code and then proceed to release the lock that we have just attained.*

A mutex lock.

when we lock something, we essentially prevent someone else coming in and messing with the locked resource whilst it’s in use.

***A Simple Lock Example***

### Syntax: class asyncio.Lock

Implements a mutex lock for asyncio tasks. Not thread-safe.

An asyncio lock can be used to guarantee exclusive access to a shared resource.

The preferred way to use a Lock is an async with statement:

In [ ]:
import asyncio
import time

async def myWorker(lock):
    print("Attempting to attain lock")
    ## acquire lock
    with await lock:
        ## run critical section of code
        print("Currently Locked")
        time.sleep(2)
    ## our worker releases lock at this poit
    print("Unlocked Critical Section")

async def main():
    ## instantiate our lock
    lock = asyncio.Lock()
    ## await the execution of 2 myWorker coroutines
    ## each with our same lock instance passed in
    await asyncio.wait([myWorker(lock), myWorker(lock)])

## Start up a simple loop and run our main function
## until it is complete
lock = asyncio.Lock()
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
print("All Tasks Completed")
print("loop close")

Attempting to attain lock
Currently Locked


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: 'with await lock' is deprecated use 'async with lock' instead
  import sys


Unlocked Critical Section
Attempting to attain lock
Currently Locked
Unlocked Critical Section
Task Executed
All Tasks Completed
loop close


### ***Event***

An event object.

### Syntax: class asyncio.Event

An event object. Not thread-safe.

An asyncio event can be used to notify multiple asyncio tasks that some event has happened.

An Event object manages an internal flag that can be set to true with the set() method and reset to false with the clear() method. The wait() method blocks until the flag is set to true. The flag is set to false initially.

In [ ]:
import asyncio,time
async def waiter(event):
    print('waiting for it ...')
    await event.wait()
    print('... got it!')

async def main():
    # Create an Event object.
    event = asyncio.Event()

    # Spawn a Task to wait until 'event' is set.
    waiter_task = asyncio.create_task(waiter(event))

    # Sleep for 1 second and set the event.
    await asyncio.sleep(1)
    event.set()

    # Wait until the waiter task is finished.
    await waiter_task

asyncio.run(main())

waiting for it ...
... got it!


coroutine wait()
Wait until the event is set.

If the event is set, return True immediately. Otherwise block until another task calls set().

set()
Set the event.

All tasks waiting for event to be set will be immediately awakened.

clear()
Clear (unset) the event.

Tasks awaiting on wait() will now block until the set() method is called again.

is_set()
Return True if the event is set.

### ***Condition***

A condition object.

### Syntax: class asyncio.Condition(lock=None)

A Condition object. Not thread-safe.

An asyncio condition primitive can be used by a task to wait for some event to happen and then get exclusive access to a shared resource.

In essence, a Condition object combines the functionality of an Event and a Lock. It is possible to have multiple Condition objects share one Lock, which allows coordinating exclusive access to a shared resource between different tasks interested in particular states of that shared resource.

The optional lock argument must be a Lock object or None. In the latter case a new Lock object is created automatically.

### Example

cond = asyncio.Condition()


async with cond:

    await cond.wait()



***which is equivalent to:***



In [ ]:
# THIS IS A SAMPLE INTENDATION FUNCTION OF HOW TO RUN AWATE

# cond = asyncio.Condition()

# ... later
# await cond.acquire()
# try:
    # await cond.wait()
# finally:
    # cond.release()

coroutine acquire()
Acquire the underlying lock.

This method waits until the underlying lock is unlocked, sets it to locked and returns True.

### ***Semaphore***

A semaphore.

### Syntax: class asyncio.Semaphore(value=1)

A Semaphore object. Not thread-safe.

A semaphore manages an internal counter which is decremented by each acquire() call and incremented by each release() call. The counter can never go below zero; when acquire() finds that it is zero, it blocks, waiting until some task calls release().

The optional value argument gives the initial value for the internal counter (1 by default). If the given value is less than 0 a ValueError is raised.

***Example***

In [ ]:
import asyncio
import time

async def myWorker(semaphore):
    await semaphore.acquire()
    print("Successfully acquired the semaphore")
    await asyncio.sleep(3)
    print("Releasing Semaphore")
    semaphore.release()

async def main(loop):
    mySemaphore = asyncio.Semaphore(value=2)
    await asyncio.wait([myWorker(mySemaphore), myWorker(mySemaphore), myWorker(mySemaphore)])
    print("Main Coroutine")

loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))
print("All Workers Completed")
print("loop close")

Successfully acquired the semaphore
Successfully acquired the semaphore
Putting news item onto queue
Consumer: 2 consumed article with id: 1
Consumer: 2 Attempting to get from queue
Task Executed
Putting news item onto queue
Consumer: 1 consumed article with id: 5
Consumer: 1 Attempting to get from queue
Task Executed
Putting news item onto queue
Consumer: 2 consumed article with id: 1
Consumer: 2 Attempting to get from queue
Task Executed
Releasing Semaphore
Successfully acquired the semaphore
Releasing Semaphore
Putting news item onto queue
Consumer: 1 consumed article with id: 3
Consumer: 1 Attempting to get from queue
Task Executed
Putting news item onto queue
Consumer: 2 consumed article with id: 1
Consumer: 2 Attempting to get from queue
Task Executed
Putting news item onto queue
Consumer: 1 consumed article with id: 3
Consumer: 1 Attempting to get from queue
Task Executed
Releasing Semaphore
Main Coroutine
All Workers Completed
loop close


### ***BoundedSemaphore***

A bounded semaphore.

### class asyncio.BoundedSemaphore(value=1)

A bounded semaphore object. Not thread-safe.

Bounded Semaphore is a version of Semaphore that raises a ValueError in release() if it increases the internal counter above the initial value.

# ***Exceptions***

### ***asyncio.TimeoutError***

Raised on timeout by functions like wait_for(). Keep in mind that asyncio.TimeoutError is unrelated to the built-in TimeoutError exception.

### Syntax: exception asyncio.TimeoutError

The operation has exceeded the given deadline.

Important This exception is different from the builtin TimeoutError exception.

### ***asyncio.CancelledError***

Raised when a Task is cancelled. See also Task.cancel().

### Syntax: exception asyncio.CancelledError

The operation has been cancelled.

This exception can be caught to perform custom operations when asyncio Tasks are cancelled. In almost all situations the exception must be re-raised.

***The following example illustrates how coroutines can intercept the cancellation request:***

In [ ]:
import asyncio,time
async def cancel_me():
    print('cancel_me(): before sleep')

    try:
        # Wait for 1 hour
        await asyncio.sleep(3600)
    except asyncio.CancelledError:
        print('cancel_me(): cancel sleep')
        raise
    finally:
        print('cancel_me(): after sleep')

async def main():
    # Create a "cancel_me" Task
    task = asyncio.create_task(cancel_me())

    # Wait for 1 second
    await asyncio.sleep(1)

    task.cancel()
    try:
        await task
    except asyncio.CancelledError:
        print("main(): cancel_me is cancelled now")

asyncio.run(main())

cancel_me(): before sleep
cancel_me(): cancel sleep
cancel_me(): after sleep
main(): cancel_me is cancelled now


# ***Excercise***

***1. Awaiting on a coroutine. The following snippet of code will print “hello” after waiting for 5 second, and then print “world” after waiting for another 7 seconds:***

In [ ]:
import asyncio
import time

async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")

    await say_after(5, 'hello')
    await say_after(7, 'world')

    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())

started at 05:12:28
hello
world
finished at 05:12:40


***2. Write a program. we are going to create a newsProducer() coroutine and a newsConsumer() coroutine. The newsProducer() coroutine will push new news items onto our synchronized queue, the newsConsumer() coroutine will attempt to retrieve any items that have been pushed onto said queue and then print whenever it does get something.***

In [ ]:
import asyncio
import random
import time

async def newsProducer(myQueue):
    while True:
        await asyncio.sleep(1)
        print("Putting news item onto queue")
        await myQueue.put(random.randint(1,5))

async def newsConsumer(id, myQueue):
    print(myQueue)
    while True:
        print("Consumer: {} Attempting to get from queue".format(id))
        item = await myQueue.get()
        if item is None:
            ## the producer emits None to indicate that it is done
            break
        print("Consumer: {} consumed article with id: {}".format(id, item))


loop = asyncio.get_event_loop()
myQueue = asyncio.Queue(loop=loop, maxsize=10)
try:
    loop.run_until_complete(asyncio.gather(newsProducer(myQueue), newsConsumer(1, myQueue), newsConsumer(2, myQueue)))
except KeyboardInterrupt:
    pass
finally:
    print("close loop")


***3. how we can use a task generator function in order to generate 5 distinct tasks for our event loop to process.***

In [ ]:
import asyncio
import time

async def myTask():
    time.sleep(1)
    print("Processing Task")

async def myTaskGenerator():
    for i in range(5):
        asyncio.ensure_future(myTask())

loop = asyncio.get_event_loop()
loop.run_until_complete(myTaskGenerator())
print("Completed All Tasks")
print("loop close")

Completed All Tasks
loop close


### ***4. Create a as_completed() function***

In [ ]:
import asyncio

async def myWorker(number):
    return number * 2

async def main(coros):
    for fs in asyncio.as_completed(coros):
        print(await fs)

coros = [myWorker(1) for i in range(5)]

try:
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main(coros))
except KeyboardInterrupt:
    pass
finally:
    print("close loop")

2
2
2
2
2
close loop


### ***5. Create simple run_until_complete() method.***

In [ ]:
import asyncio
import time

async def myWork():
    print("Starting Work")
    time.sleep(5)
    print("Finishing Work")

loop = asyncio.get_event_loop()
try:
    loop.run_until_complete(myWork())
finally:
    print("close loop")

Starting Work
Finishing Work
close loop
Task Executed
Putting news item onto queue
Consumer: 2 consumed article with id: 4
Consumer: 2 Attempting to get from queue


### ***6. Create simple run_forever() method***

In [ ]:
import asyncio

async def work():
    while True:
        await asyncio.sleep(1)
        print("Task Executed")

loop = asyncio.get_event_loop()
try:
    asyncio.ensure_future(work())
    loop.run_forever()
except KeyboardInterrupt:
    pass
finally:
    print("Closing Loop")
    

### ***7. Create Running Multiple coroutines:***

In [ ]:
import asyncio
import time

async def firstWorker():
    while True:
        await asyncio.sleep(1)
        print("First Worker Executed")

async def secondWorker():
    while True:
        await asyncio.sleep(1)
        print("Second Worker Executed")


loop = asyncio.get_event_loop()
try:
    asyncio.ensure_future(firstWorker())
    asyncio.ensure_future(secondWorker())
    loop.run_forever()
except KeyboardInterrupt:
    pass
finally:
    print("Closing Loop")
    # loop.close()

### ***8. Create cancel() function***

In [ ]:
import asyncio
import time

async def myTask():
    time.sleep(1)
    print("Processing Task")

    for task in asyncio.Task.all_tasks():
        print(task)
        task.cancel()
        print(task)

async def main():
    for i in range(5):
        asyncio.ensure_future(myTask())

loop = asyncio.get_event_loop()
loop.run_until_complete(main())
print("Completed All Tasks")


Completed All Tasks


### ***9. Create a Asyncio Futures***

In [ ]:
import asyncio

## Define a coroutine that takes in a future
async def myCoroutine(future):
    ## simulate some 'work'
    await asyncio.sleep(1)
    ## set the result of our future object
    future.set_result("My Coroutine-turned-future has completed")

async def main():
    ## define a future object
    future = asyncio.Future()
    ## wait for the completion of our coroutine that we've
    ## turned into a future object using the ensure_future() function
    await asyncio.ensure_future(myCoroutine(future))
    ## Print the result of our future
    print(future.result())

## Spin up a quick and simple event loop
## and run until completed
loop = asyncio.get_event_loop()
try:
    loop.run_until_complete(main())
finally:
    print("close loop")

My Coroutine-turned-future has completed
close loop


### ***10 . Create Multiple Coroutines***

In [ ]:
import asyncio
import random

async def myCoroutine(id):
    process_time = random.randint(1,5)
    await asyncio.sleep(process_time)
    print("Coroutine: {}, has successfully completed after {} seconds".format(id, process_time))

async def main():
    tasks = []
    for i in range(10):
        tasks.append(asyncio.ensure_future(myCoroutine(i)))

    await asyncio.gather(*tasks)


loop = asyncio.get_event_loop()
try:
    loop.run_until_complete(main())
finally:
    print("close loop")

Coroutine: 3, has successfully completed after 1 seconds
Coroutine: 5, has successfully completed after 1 seconds
Coroutine: 8, has successfully completed after 1 seconds
Coroutine: 9, has successfully completed after 1 seconds
Coroutine: 1, has successfully completed after 2 seconds
Coroutine: 7, has successfully completed after 2 seconds
Coroutine: 4, has successfully completed after 4 seconds
Coroutine: 6, has successfully completed after 4 seconds
Coroutine: 0, has successfully completed after 5 seconds
Coroutine: 2, has successfully completed after 5 seconds
close loop
